In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import os
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import pdb

In [4]:
import cv2

In [5]:
import skimage.feature

In [6]:
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose,Permute,Reshape,Activation
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K

Using TensorFlow backend.


In [7]:
from keras.utils.np_utils import to_categorical

In [8]:
import random

In [9]:
K.set_image_dim_ordering('tf')

In [10]:
TRAIN_PATH = '/Users/chiyuan/Documents/learning/kaggle/sea_lion/TrainSmall2/Train/'
TRAIN_GT_PATH = '/Users/chiyuan/Documents/learning/kaggle/sea_lion/TrainSmall2/TrainDotted/'
TRAIN_FILENAMES = [item for item in os.listdir(TRAIN_PATH) if not item.startswith('.')]
TRAIN_FILENAMES = sorted(TRAIN_FILENAMES, key = lambda item: int(item.partition('.')[0]))

In [11]:
TRAIN_FILENAMES

['41.jpg',
 '42.jpg',
 '43.jpg',
 '44.jpg',
 '45.jpg',
 '46.jpg',
 '47.jpg',
 '48.jpg',
 '49.jpg',
 '50.jpg']

In [12]:
num_images = len(TRAIN_FILENAMES)
image_row, image_col = 512, 512

In [13]:
num_cat = 5
def generate_image_and_gt(TRAIN_FILENAMES,TRAIN_PATH,TRAIN_GT_PATH,batch_size):
    i = 0
    while True:
        random.shuffle(TRAIN_FILENAMES)
        imgs = np.ndarray((batch_size,image_row,image_col,3),dtype=np.float32)
        imgs_gt = np.ndarray((batch_size,image_row,image_col,num_cat),dtype=np.uint8)
        for TRAIN_FILENAME in TRAIN_FILENAMES:
            # in ground truth data, black out
            # in ground truth data, black out

            image = cv2.imread(TRAIN_PATH+TRAIN_FILENAME)
            #image = cv2.resize(image,(image_row,image_col),interpolation = cv2.INTER_AREA)
            image_gt = cv2.imread(TRAIN_GT_PATH+TRAIN_FILENAME)
            #image_gt = cv2.resize(image,(image_row,image_col),interpolation = cv2.INTER_AREA)

            image_mask = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
            image_mask[image_mask<20] = 0 
            image_mask[image_mask>20] = 255

            image_gt_mask = cv2.cvtColor(image_gt,cv2.COLOR_BGR2GRAY)
            image_gt_mask[image_gt_mask<20] = 0 
            image_gt_mask[image_gt_mask>20] = 255

            image_diff = cv2.absdiff(image,image_gt)
            image_diff = cv2.bitwise_or(image_diff,image_diff,mask=image_gt_mask)
            image_diff = cv2.bitwise_or(image_diff,image_diff,mask=image_mask)
            image_diff = cv2.cvtColor(image_diff,cv2.COLOR_BGR2GRAY)

            blobs = skimage.feature.blob_log(image_diff,min_sigma=3,max_sigma=4,num_sigma=1,threshold=0.02)
            gt_data = np.zeros((image.shape[0],image.shape[1],5))
            # gt_adult_male = np.zeros(image.shape)
            # gt_subadult_male = np.zeros(image.shape)
            # gt_pups = np.zeros(image.shape)
            for blob in blobs:
                y,x,s = blob
                b,g,r = image[int(y)][int(x)][:]
                if r > 200 and b < 50 and g < 50: # RED adult male
                    gt_data[int(y),int(x),0] = 1  
                elif r > 200 and b > 200 and g < 50: # MAGENTA subadult_males
                    gt_data[int(y),int(x),1] = 1         
                elif r < 100 and b < 100 and 150 < g < 200: # GREEN pups
                    gt_data[int(y),int(x),2] = 1
                elif r < 100 and  100 < b and g < 100: # BLUE juveniles
                    gt_data[int(y),int(x),3] = 1
                elif r < 150 and b < 50 and g < 100:  # BROWN adult_females
                    gt_data[int(y),int(x),4] = 1
            image = cv2.resize(image,(image_row,image_col),interpolation = cv2.INTER_AREA)
            for cat in range(num_cat):
                 imgs_gt[i,:,:,cat] = cv2.resize(gt_data[:,:,cat],(image_row,image_col))
            #image = image.astype('float32')
            imgs[i] = image#.transpose(2,0,1)
            #imgs_gt[i] = gt_data
            if i+1 == batch_size:
                imgs-=np.mean(imgs)
                imgs/=np.std(imgs)
                print 'yield ',TRAIN_FILENAME
                yield(imgs,imgs_gt.reshape(-1,image_row*image_col,num_cat))
                imgs = np.ndarray((batch_size,image_row,image_col,3),dtype=np.float32)
                imgs_gt = np.ndarray((batch_size,image_row,image_col,num_cat),dtype=np.uint8)
                i = 0
            else:
                i += 1

            #need to create a iternator if can't feet all images? 
    
    

In [25]:
# from keras.utils import to_categorical
# y_train = to_ccategorical(np.random.randint(10, size=(1000, 1)), num_classes=10)

In [14]:
smooth = 1.
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)
def get_unet():
    inputs = Input((image_row, image_col, 3))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
    
    conv10 = Conv2D(num_cat, (1, 1))(inputs)
    reshape1 = Reshape((image_row*image_col,num_cat))(conv10)
    soft = Activation('softmax')(reshape1)
    model = Model(inputs=[inputs], outputs=[soft])
    
    model.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [128]:
# i = 0

# imgs = np.ndarray((num_images,image_row,image_col,3),dtype=np.float32)
# imgs_gt = np.ndarray((num_images,image_row,image_col,num_cat),dtype=np.float32)
# for TRAIN_FILENAME in TRAIN_FILENAMES:
#     # in ground truth data, black out

#     image = cv2.imread(TRAIN_PATH+TRAIN_FILENAME)
#     #image = cv2.resize(image,(image_row,image_col),interpolation = cv2.INTER_AREA)
#     image_gt = cv2.imread(TRAIN_GT_PATH+TRAIN_FILENAME)
#     #image_gt = cv2.resize(image,(image_row,image_col),interpolation = cv2.INTER_AREA)

#     image_mask = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
#     image_mask[image_mask<20] = 0 
#     image_mask[image_mask>20] = 255

#     image_gt_mask = cv2.cvtColor(image_gt,cv2.COLOR_BGR2GRAY)
#     image_gt_mask[image_gt_mask<20] = 0 
#     image_gt_mask[image_gt_mask>20] = 255

#     image_diff = cv2.absdiff(image,image_gt)
#     image_diff = cv2.bitwise_or(image_diff,image_diff,mask=image_gt_mask)
#     image_diff = cv2.bitwise_or(image_diff,image_diff,mask=image_mask)
#     image_diff = cv2.cvtColor(image_diff,cv2.COLOR_BGR2GRAY)

#     blobs = skimage.feature.blob_log(image_diff,min_sigma=3,max_sigma=4,num_sigma=1,threshold=0.02)
#     gt_data = np.zeros((image.shape[0],image.shape[1],5))
#     # gt_adult_male = np.zeros(image.shape)
#     # gt_subadult_male = np.zeros(image.shape)
#     # gt_pups = np.zeros(image.shape)
#     for blob in blobs:
#         y,x,s = blob
#         b,g,r = image[int(y)][int(x)][:]
#         if r > 200 and b < 50 and g < 50: # RED adult male
#             gt_data[int(y),int(x),0] = 1  
#         elif r > 200 and b > 200 and g < 50: # MAGENTA subadult_males
#             gt_data[int(y),int(x),1] = 1         
#         elif r < 100 and b < 100 and 150 < g < 200: # GREEN pups
#             gt_data[int(y),int(x),2] = 1
#         elif r < 100 and  100 < b and g < 100: # BLUE juveniles
#             gt_data[int(y),int(x),3] = 1
#         elif r < 150 and b < 50 and g < 100:  # BROWN adult_females
#             gt_data[int(y),int(x),4] = 1
#     image = cv2.resize(image,(image_row,image_col),interpolation = cv2.INTER_AREA)
#     for cat in range(num_cat):
#          imgs_gt[i,:,:,cat] = cv2.resize(gt_data[:,:,cat],(image_row,image_col))
#     #image = image.astype('float32')
#     imgs[i] = image#.transpose(2,0,1)
#     #imgs_gt[i] = gt_data
#     i += 1
# imgs-=np.mean(imgs)
# imgs/=np.std(imgs)

In [14]:
#imgs_gt_reshape = imgs_gt.reshape(-1,image_row*image_col,num_cat)

NameError: name 'imgs_gt' is not defined

In [15]:
model = get_unet()
# model_fit = model.fit(imgs, imgs_gt_reshape, batch_size=32, nb_epoch=20, verbose=1, shuffle=True,
#                validation_split=0.1)
# #model.train_on_batch(imgs[:,:,:,[0]],imgs_gt[...,np.newaxis])

In [16]:
TRAIN_PATH2 = '/Users/chiyuan/Documents/learning/kaggle/sea_lion/KaggleNOAASeaLions/Train/'
TRAIN_GT_PATH2 = '/Users/chiyuan/Documents/learning/kaggle/sea_lion/KaggleNOAASeaLions/TrainDotted/'
TRAIN_FILENAMES2 = [item for item in os.listdir(TRAIN_PATH2) if not item.startswith('.')]
random.seed(1)
train_len_tot = len(TRAIN_FILENAMES2)
random.shuffle(TRAIN_FILENAMES2)
SPLIT = 0.8

In [17]:
train_len = int(train_len_tot*SPLIT)
val_len = int(train_len_tot*(1-SPLIT))
TRAIN_FILENAME_TRAIN=TRAIN_FILENAMES2[:train_len]
TRAIN_FILENAME_VAL=TRAIN_FILENAMES2[train_len:]
TEST_PATH = '/Users/chiyuan/Documents/learning/kaggle/sea_lion/KaggleNOAASeaLions/Test/'
TEST_FILENAME = [item for item in os.listdir(TEST_PATH) if not item.startswith('.')]

In [18]:
batch_size=2
train_generator = generate_image_and_gt(TRAIN_FILENAME_TRAIN,TRAIN_PATH2,TRAIN_GT_PATH2,batch_size=batch_size)
val_generator = generate_image_and_gt(TRAIN_FILENAME_VAL,TRAIN_PATH2,TRAIN_GT_PATH2,batch_size=batch_size)
model.fit_generator(train_generator, 
                    steps_per_epoch=5, 
                    epochs=1,
                    #validation_data=val_generator,
                    #validation_steps=val_len//batch_size
                   )
#model.save_weights("sea_lion_count.h5")

Epoch 1/1
yield  665.jpg
5/5 [==============================] - 71s - loss: 0.0000e+00 - acc: 0.4517     
yield  666.jpg


In [21]:
predict_train = model.predict_generator(train_generator,2)

yield  195.jpg
yield  616.jpg
yield  651.jpg


In [23]:
predict_train

array([[[ 0.35289609,  0.15443744,  0.26292479,  0.12204544,  0.10769621],
        [ 0.34160167,  0.15535995,  0.25667459,  0.13065055,  0.11571325],
        [ 0.43500885,  0.12122833,  0.28171253,  0.08394188,  0.07810835],
        ..., 
        [ 0.24546218,  0.20358321,  0.24095453,  0.1469174 ,  0.1630827 ],
        [ 0.23010258,  0.20218502,  0.22876491,  0.16017143,  0.17877604],
        [ 0.23744   ,  0.20424716,  0.23830767,  0.1473209 ,  0.17268425]],

       [[ 0.47647429,  0.09694506,  0.29702857,  0.05601815,  0.07353386],
        [ 0.48623556,  0.09607685,  0.29657483,  0.05466431,  0.06644838],
        [ 0.48644873,  0.09592693,  0.30080158,  0.05126247,  0.06556026],
        ..., 
        [ 0.19783199,  0.22322848,  0.22431791,  0.15281299,  0.20180863],
        [ 0.20012204,  0.22909155,  0.22708438,  0.15189308,  0.19180889],
        [ 0.20865662,  0.2107048 ,  0.22902113,  0.14689575,  0.20472161]],

       [[ 0.17393167,  0.16854611,  0.15285924,  0.29958197,  0.2050

In [163]:
train_generator = generate_image_and_gt(TRAIN_FILENAMES,batch_size=2)
for x,y in train_generator:
    print x.shape
    print y.shape

41.jpg
42.jpg
1 yield
(2, 512, 512, 3)
(2, 262144, 5)
43.jpg
44.jpg
1 yield
(2, 512, 512, 3)
(2, 262144, 5)
45.jpg
46.jpg
1 yield
(2, 512, 512, 3)
(2, 262144, 5)
47.jpg
48.jpg
1 yield
(2, 512, 512, 3)
(2, 262144, 5)
49.jpg
50.jpg
1 yield
(2, 512, 512, 3)
(2, 262144, 5)


In [117]:
%debug

> /Users/chiyuan/anaconda/lib/python2.7/site-packages/ipykernel/kernelbase.py(719)_input_request()
    717             except KeyboardInterrupt:
    718                 # re-raise KeyboardInterrupt, to truncate traceback
--> 719                 raise KeyboardInterrupt
    720             else:
    721                 break

ipdb> q


In [130]:
imgs.shape

(10, 512, 512, 3)

In [131]:
imgs_gt.shape

(10, 512, 512, 5)

In [133]:
imgs_gt_reshape.shape

(10, 262144, 5)

In [122]:
to_categorical(imgs_gt[...,np.newaxis]).shape

(2621440, 1)